#IIC-2433 Minería de Datos UC

* Versiones de librerías, python 3.8.10
* pandas 1.5.3
* tensorflow/keras 2.12.0

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# 1. Carga de la base de datos

En esta tarea se trabajará con un dataset de vinos obtenido de Kaggle:
Cárguelo, léalo y muéstrelo.

https://www.kaggle.com/datasets/yasserh/wine-quality-dataset

In [2]:
dframe = pd.read_csv("WineQT.csv", encoding = "ISO-8859-1")
dframe.shape

(1143, 13)

In [3]:
dframe

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


# 2. Preprocesamiento

Realice el preprocesamiento que considere adecuado para este *dataset* y argumente todas sus decisiones.

In [34]:
 #Chequeamos si hay algun valor nulo en el df, y si hay, eliminamos la fila
if dframe.isnull().values.any():
    dframe.dropna()

In [35]:
#Se hace un robust scaling para eliminar outliers

Q1 = dframe.quantile(0.25)
Q3 = dframe.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

filtered_df = dframe[~((dframe < lower_bound) | (dframe > upper_bound)).any(axis=1)]
filtered_df


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


In [36]:
# predeciremos Quality

X = filtered_df.drop('quality', axis=1)
y = filtered_df['quality']
y = to_categorical(y)

# 3. Preguntas iniciales


Deberá encontrar la mejor combinación de hiperparámetros que le permita obtener el modelo con mejor *accuracy*.

A priori, ¿cuáles cree que serán las mejores combinaciones de hiperparámetros para este problema en particular? ¿Por qué?

- Un patience alto puede ayudar a obtener un mejor modelo, dado que define cuantos epochs consecutivos pueden haber sin mejora, pero si despues logra "desestancarse" seguira mejorando 
- un alto numero de epochs combinado con el parametro que dije antes.
- El numero de capas neuronales para aumentar la complejidad del modelo (aunque a su vez afecta su velocidad y uso de recursos)

# 3. Modelos

Ocupando la librería **keras** de tensorflow, construya redes neuronales multicapas variando los siguientes hiperparámetros: función de activación, cantidad de neuronas por cada capa, cantidad de capas, optimizador y cantidad de épocas.

### INTENTO 1

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

model.fit(X_train, y_train, validation_split=0.2, epochs=300, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/300
17/17 [==============================] - 0s 8ms/step - loss: 2.1654 - accuracy: 0.0713 - val_loss: 1.9485 - val_accuracy: 0.2090
Epoch 2/300
17/17 [==============================] - 0s 2ms/step - loss: 1.7957 - accuracy: 0.4165 - val_loss: 1.6203 - val_accuracy: 0.5000
Epoch 3/300
17/17 [==============================] - 0s 2ms/step - loss: 1.4612 - accuracy: 0.5572 - val_loss: 1.2757 - val_accuracy: 0.5373
Epoch 4/300
17/17 [==============================] - 0s 2ms/step - loss: 1.1581 - accuracy: 0.5985 - val_loss: 1.0418 - val_accuracy: 0.6045
Epoch 5/300
17/17 [==============================] - 0s 2ms/step - loss: 0.9803 - accuracy: 0.6154 - val_loss: 0.9456 - val_accuracy: 0.6194
Epoch 6/300
17/17 [==============================] - 0s 2ms/step - loss: 0.9030 - accuracy: 0.6417 - val_loss: 0.9130 - val_accuracy: 0.6119
Epoch 7/300
17/17 [==============================] - 0s 2ms/step - loss: 0.8672 - accuracy: 0.6454 - val_loss: 0.9034 - val_accuracy: 0.6343
Epoch 8/300
1

In [38]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.6776611694152923

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        17
           5       0.68      0.80      0.74       297
           6       0.67      0.62      0.64       280
           7       0.69      0.56      0.62        73

    accuracy                           0.68       667
   macro avg       0.51      0.50      0.50       667
weighted avg       0.66      0.68      0.67       667



Lo logre altiro xd

### INTENTO 2

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.fit(X_train, y_train, validation_split=0.2, epochs=200, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/200
15/15 [==============================] - 1s 12ms/step - loss: 1.9869 - accuracy: 0.2575 - val_loss: 1.7848 - val_accuracy: 0.4188
Epoch 2/200
15/15 [==============================] - 0s 3ms/step - loss: 1.5556 - accuracy: 0.4549 - val_loss: 1.3433 - val_accuracy: 0.4188
Epoch 3/200
15/15 [==============================] - 0s 3ms/step - loss: 1.1921 - accuracy: 0.5215 - val_loss: 1.0505 - val_accuracy: 0.5641
Epoch 4/200
15/15 [==============================] - 0s 3ms/step - loss: 1.0139 - accuracy: 0.6094 - val_loss: 0.9726 - val_accuracy: 0.5812
Epoch 5/200
15/15 [==============================] - 0s 3ms/step - loss: 0.9535 - accuracy: 0.6116 - val_loss: 0.9659 - val_accuracy: 0.5726
Epoch 6/200
15/15 [==============================] - 0s 3ms/step - loss: 0.9081 - accuracy: 0.6288 - val_loss: 0.9467 - val_accuracy: 0.5812
Epoch 7/200
15/15 [==============================] - 0s 3ms/step - loss: 0.8586 - accuracy: 0.6373 - val_loss: 0.9269 - val_accuracy: 0.5812
Epoch 8/200


In [40]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.7787307032590052

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        14
           5       0.78      0.83      0.81       261
           6       0.75      0.81      0.78       241
           7       0.93      0.63      0.75        67

    accuracy                           0.78       583
   macro avg       0.62      0.57      0.58       583
weighted avg       0.77      0.78      0.77       583



### INTENTO 3

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

model.fit(X_train, y_train, validation_split=0.2, epochs=40, callbacks=callbacks)

y_train_pred = np.argmax(model.predict(X_train), axis=-1)
y_test_pred = np.argmax(model.predict(X_test), axis=-1)

y_train_label = np.argmax(y_train, axis=-1)
y_test_label = np.argmax(y_test, axis=-1)

Epoch 1/40
15/15 [==============================] - 1s 10ms/step - loss: 2.0439 - accuracy: 0.1545 - val_loss: 1.9262 - val_accuracy: 0.3419
Epoch 2/40
15/15 [==============================] - 0s 2ms/step - loss: 1.8204 - accuracy: 0.4657 - val_loss: 1.7165 - val_accuracy: 0.4615
Epoch 3/40
15/15 [==============================] - 0s 3ms/step - loss: 1.5825 - accuracy: 0.5064 - val_loss: 1.4804 - val_accuracy: 0.4786
Epoch 4/40
15/15 [==============================] - 0s 3ms/step - loss: 1.3439 - accuracy: 0.5086 - val_loss: 1.2661 - val_accuracy: 0.4872
Epoch 5/40
15/15 [==============================] - 0s 3ms/step - loss: 1.1686 - accuracy: 0.5815 - val_loss: 1.1178 - val_accuracy: 0.5726
Epoch 6/40
15/15 [==============================] - 0s 3ms/step - loss: 1.0562 - accuracy: 0.5966 - val_loss: 1.0407 - val_accuracy: 0.5726
Epoch 7/40
15/15 [==============================] - 0s 3ms/step - loss: 0.9838 - accuracy: 0.6073 - val_loss: 0.9957 - val_accuracy: 0.5726
Epoch 8/40
15/15 [=

In [42]:
print("Train Accuracy: ", accuracy_score(y_train_label, y_train_pred))

print("\nTrain Classification Report:")
print(classification_report(y_train_label, y_train_pred,zero_division=0))

Train Accuracy:  0.7272727272727273

Train Classification Report:
              precision    recall  f1-score   support

           4       0.00      0.00      0.00        14
           5       0.69      0.89      0.78       261
           6       0.80      0.59      0.68       241
           7       0.71      0.73      0.72        67

    accuracy                           0.73       583
   macro avg       0.55      0.55      0.55       583
weighted avg       0.72      0.73      0.71       583



Intente encontrar un modelo que obtenga un 66.6% de *accuracy*. ¿Lo logró? ¿por qué?

Se logro esta mision gracias a los hiperparametros que mencionamos en el principio, en donde si tenemos una gran cantidad de epochs combinados con un buen patiente, ademas de tener un buen numero de redes neuronales, pude obtener un modelo casi con un accuracy casi 10% mayor a lo que me pedian.

Una vez obtenido el mejor clasificador, ¿cuáles son sus errores más comunes? ¿cómo se podrían mejorar?

Los errores mas comunes radican en varias cosas, las mas comunes son errores dentro del dataset, en donde los datos que se tomaron para el entrenamiento no representa de manera fidegnina a la real. Tambien podemos contar con que se entreno con muchos outliers y aprendio de manera erronea como predecir los datos. Por otra parte, tambien podemos tener errores al crear el modelo, usando hiperparametros que no favorezcan al modelo. Todos estos problemas se pueden mejorar ya sea preprocesando el dataset y haciendo una busqueda de grilla para obtener los mejores parametros del modelo.

# 4. Preguntas finales

¿Qué resultados obtuvo? Se habrá dado cuenta que encontrar el mejor modelo no es lo mismo que buscar los mejores hiperparámetros unilateralmente. ¿Cuáles combinaciones funcionaron mejor y cuáles peor? ¿Por qué cree que fue así? Argumente.

Mi mejor resultado, al menos a la hora que se corrio el codigo con esta respuesta, es de un accuracy de 80%, la combinacion que se uso para lograr esto es un patiente de 10 con un red de 128, 64, 32 y 16 neuronas. en este intento la cantidad de epoch no fue necesaria que fuera alta, dado que el patiente lo detuvo en el intento 24. La combinacion que peor funciono fue cuando se usaron menos redes neuronales, dando resultados con un accuracy menor al 70%
La razon por la que el accuracy aumentan proporcionalmente con las redes neuronales es porque esta le permite aumentar la complejidad del modelo, por lo que si el dataset, como en este caso, tiene un buen procesamiento, puede ser de mucha ayuda a la hora de mejorar las predicciones.